In [111]:
import numpy as np
import copy
import matplotlib.pyplot as plt
import sys


eps=sys.float_info.epsilon

#EG con pivot

def pivot(a:np.array,i)->int:
    max=abs(a[i,i])
    res=0
    for k in range(i+1,len(a)):
        if(abs(a[k,i])>max):
            max=a[k,i]
            res=k
    return res




def eliminacion_gaussiana_pivot(m:np.array,b:np.array)->np.array:
    if(len(m)!=len(m[0])):
        print("Error. Matriz no cuadrada")

    #Construimos A' , extension de A con b
    n=len(m)
    m_ext=np.hstack((m,np.zeros((n,1))))
    m_ext[:,n]=b


    for i in range(0,n):

        fila_a_permutar=pivot(m_ext,i)
        if(fila_a_permutar>i):      #Para el caso que devuelve 0
            copy=np.array(m_ext[i,:])
            m_ext[i,:]=m_ext[fila_a_permutar,:]
            m_ext[fila_a_permutar,:]=copy


        if(np.all(m_ext[:,i+1:] == 0)):
            pass

        for j in range(i+1,n):
            if(np.abs(m_ext[i,i])<eps):
                print("Posible Error Numérico.Cociente con divisor cercano a 0.\n")
            multiplier=m_ext[j,i]/m_ext[i,i]
            m_ext[j,:]=m_ext[j,:]-m_ext[i,:]*multiplier     #M_j=M_j - M_i * multiplier

    #Ya A' escalonada -> despejamos x
    x=np.zeros(n)
    for i in range(n-1,-1,-1):
        if(np.abs(m_ext[i,i])<eps):
                print("Posible Error Numérico.Cociente con divisor cercano a 0.\n")
        b_i=m_ext[i,n]
        if(i==n-1):
            x[i]=b_i/m_ext[i,i]
            continue
        prod=np.dot(m_ext[i,i+1:n],x[i+1:n])
        x[i]=(b_i-prod)/m_ext[i,i]

    #Devolvemos X=[x1,...,xn]
    return x

#TODO:modificar para que devuelva excepcion si hay inf. soluciones o no las hay


#matrix=np.array([[0,1,1,2],
#                 [2,-1,1,0],
#                 [3,1,0,1],
#                 [0,2,1,3]],dtype=np.float32)

matrix=np.array([[3,1,2],
                [3,0,1],
                [1,0,2]],dtype=np.float32)

b=np.array([1,1,1],dtype=np.float32)


# AX0=?b
matrix@eliminacion_gaussiana_pivot(matrix,b)



array([1., 1., 1.])